00 intro

In [1]:
import numpy as np
import plotly.graph_objects as go

from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import ema_tv

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=1500,
)
timestamps = candles[:, CandleBodyType.Timestamp]
datetimes = timestamps.astype("datetime64[ms]")
open_prices = candles[:, CandleBodyType.Open]
high_prices = candles[:, CandleBodyType.High]
low_prices = candles[:, CandleBodyType.Low]
close_prices = candles[:, CandleBodyType.Close]
volume = candles[:, CandleBodyType.Volume]

In [3]:
above_ema_bool = False
falling_length = 3
ema_length = 200

In [4]:
falling_length -= 1

In [5]:
open_prices = candles[:, CandleBodyType.Open]
prev_open_prices = np.roll(open_prices, 1)
prev_open_prices[0] = np.nan

In [6]:
close_prices = candles[:, CandleBodyType.Close]
prev_close_prices = np.roll(close_prices, 1)
prev_close_prices[0] = np.nan

In [7]:
current_green_candles = open_prices < close_prices

close_diff = np.diff(close_prices, prepend=np.nan)

red_candles = close_diff < 0

In [8]:
falling_bools = np.full_like(close_diff, False, dtype=np.bool_)
for x in range(falling_length, close_diff.size):
    if red_candles[x - falling_length : x + 1].all():
        falling_bools[x] = True

falling_bools = np.roll(falling_bools, 1)
falling_bools[0] = False

In [9]:
ema = ema_tv(source=close_prices, length=ema_length)

if above_ema_bool:
    above_below_bool = close_prices > ema
else:
    above_below_bool = close_prices < ema

bullish_engulfing_candles = (close_prices > prev_open_prices) & (open_prices <= prev_close_prices)

entries = current_green_candles & bullish_engulfing_candles & falling_bools & above_below_bool

entry_signals = np.where(entries, close_prices, np.nan)

exit_prices = np.full_like(close_prices, np.nan)

In [10]:
fig = go.Figure()
fig.add_candlestick(
    x=datetimes,
    open=open_prices,
    high=high_prices,
    low=low_prices,
    close=close_prices,
    name="Candles",
)
fig.add_scatter(
    x=datetimes,
    y=ema,
    line_color="aqua",
)
fig.add_scatter(
    x=datetimes,
    y=entry_signals,
    name="Signals",
    mode="markers",
    marker=dict(
        size=12,
        symbol="hexagram",
        color="yellow",
        line=dict(width=1, color="DarkSlateGrey"),
    ),
)
fig.update_layout(height=800, xaxis_rangeslider_visible=False)
fig.show()